##### To submit jobs for terrain correction to Alaska Satellite Facility's HyP3 API

In [19]:
import pandas as pd
import os
import hyp3_sdk as sdk
from hyp3_sdk import HyP3
import getpass

In [ ]:
uid = getpass.getpass("Enter your EathData user id?")
pswd = getpass.getpass("Enter your EathData password?")
hyp3 = HyP3(username=uid, password=pswd)

In [4]:
DATA_DIR = "./data/files_list"
s1_an_df = pd.read_csv(os.path.join(DATA_DIR, "s1_an.csv"), names=["ID","Path"])

In [16]:
PROJECT_NAME = 'Antarctica-coreg-test-20m-2024'
SCENE_LIST = [os.path.splitext(os.path.basename(s))[0] for s in s1_an_df.Path[s1_an_df.ID == "056432_06EA69"] if "SLC" in s]
DOWNLOAD_FOLDER = "./data/asf/"

In [ ]:
# submit jobs
rtc_jobs = sdk.Batch()
for g in SCENE_LIST:
    rtc_jobs += hyp3.submit_rtc_job(g, 
                                    include_dem=True, #include dem in final product
                                    include_inc_map=True,  #include dem map in final product
                                    include_rgb=True,  #include rgb img in final product
                                    include_scattering_area=True, #include scat area in final product
                                    name=PROJECT_NAME,
                                    resolution=20,
                                    dem_name='copernicus',
                                    radiometry='gamma0')
print(rtc_jobs)

In [ ]:
# watch jobs
rtc_jobs = hyp3.find_jobs(name=PROJECT_NAME)
rtc_jobs = hyp3.watch(rtc_jobs)

In [ ]:
# check jobs completed
print(PROJECT_NAME)
rtc_jobs = hyp3.find_jobs(name=PROJECT_NAME)
rtc_jobs = hyp3.refresh(rtc_jobs)
running_jobs = rtc_jobs.filter_jobs(succeeded=False, running=True, failed=False)
print(f'Number of running jobs: {len(running_jobs)}')
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = rtc_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

In [ ]:
# download jobs
download_jobs = succeeded_jobs
for i,job in enumerate(download_jobs):
    print(f'scene {i+1} of {len(download_jobs)}')
    # download
    file_list = job.download_files(DOWNLOAD_FOLDER)
    process_time = job.processing_times